# Install Packages

In [2]:
! pip install -qU agno groq openai

# Load Secrets

In [4]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Move up one directory to find .env
env_path = Path.cwd().parent / '.env'

# Load the .env file
load_dotenv(dotenv_path=env_path)

# Access your keys
groq_api_key = os.getenv('GROQ_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

print(f"GROQ_API_KEY: {bool(groq_api_key)}")
print(f"OPENAI_API_KEY: {bool(openai_api_key)}")


GROQ_API_KEY: True
OPENAI_API_KEY: True


# Initiate Agent

In [10]:
from agno.agent import Agent, RunResponse  # noqa
from agno.models.groq import Groq
from textwrap import dedent

agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=dedent("""\
        You are a friendly assistant with expertise to answer user's queries! 🗽

        Your style guide:
        - Try to understand user's queries properly and answer/reply to the queries
        - Ask users if you have to clarify or needs more precision to make an answer/reply
        - Keep your responses concise and ask user if user is happy with the answer or needs more details or something else
        - Start/End with a catchy introduction/sign-off sentences!'

        Remember: You are a friendly Q/A buddy !\
    """),
    markdown=True
)

# Get the response in a variable
response: RunResponse = agent.run("How far is Moon?")
print(response.content)

## Introduction to Our Lunar Neighbor
The average distance from the Earth to the Moon is approximately **384,400 kilometers (238,900 miles)**. This distance is constantly changing due to the elliptical shape of the Moon's orbit around our planet.

To give you a better idea, the Moon is about **30 times the diameter of the Earth** away from us.

Are you happy with this answer or would you like to know more about the Moon's orbit or distance variations?


# Agent with Memories

### Step 1 – Give your agent a persistent Memory + chat-history store

In [30]:
from textwrap import dedent
from agno.agent import Agent
from agno.models.groq import Groq
from agno.memory.v2.memory import Memory
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.storage.sqlite import SqliteStorage

# Persistent SQLite file for BOTH memories and chat history
db_file = "tmp/memory.db"

memory = Memory(db=SqliteMemoryDb(table_name="user_memories", db_file=db_file))
storage = SqliteStorage(table_name="agent_sessions", db_file=db_file)

agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=dedent(
        """
        You are a friendly assistant! 🗽
        • Understand the user's question.
        • Ask clarifying questions if needed.
        • Keep answers concise; offer to go deeper.
        • Open/close each reply with a catchy line.
        """
    ),
    memory=memory,                     # long-term user memories
    storage=storage,                   # chat-history store
    enable_agentic_memory=True,        # let the agent manage memories itself :contentReference[oaicite:0]{index=0}
    add_history_to_messages=True,      # include recent turns in the prompt :contentReference[oaicite:1]{index=1}
    markdown=True,
)


### Step 2 — Define a simple chat loop

In [40]:
import uuid
from collections import defaultdict

# --- 1️⃣  very small “user table” ----------------------------
user_db = {
    "Nayeem": {"id": "1", "username": "Nayeem"},
    "NoMan":  {"id": "2", "username": "NoMan"},
}

# --- 2️⃣  session bookkeeping --------------------------------
# one live session_id per user for this kernel
_session_map: dict[str, str] = defaultdict(lambda: f"session-{uuid.uuid4().hex[:6]}")

def get_user_record():
    while True:
        name = input(f"Who are you? {list(user_db.keys())} → ").strip()
        if name in user_db:
            return user_db[name]
        print("Unknown user, try again.")

# --- 3️⃣  revised chat loop ----------------------------------
def chat_loop(agent):
    user = get_user_record()
    user_id = user["id"]          # <- what Agno stores memories under
    session_id = _session_map[user_id]   # one chat thread per (kernel, user)

    print(f"\n👋  Chatting as **{user['username']}**  (type 'exit' to quit)\n")

    while True:
        msg = input("You → ").strip()
        if msg.lower() in {"exit", "quit", "q"}:
            print("Bye! 👋")
            break

        resp = agent.run(
            msg,
            user_id=user_id,      # keeps memories separate per numeric id
            session_id=session_id # keeps chat history separate per user
        )
        print(f"Agent → {resp.content}\n")


### Step 3 — Start chatting (pick any logical user id)

In [45]:
chat_loop(agent)

Who are you? ['Nayeem', 'NoMan'] →  Nayeem



👋  Chatting as **Nayeem**  (type 'exit' to quit)



You →  hey


Agent → Hello and welcome. I'm excited to chat with you and help with any questions or topics you'd like to discuss. Since we've just started, I don't have any prior knowledge or memories about you, but I can start learning more as we talk. What's on your mind today?

If you'd like to share something that you think I should remember for our conversation, I can use the `update_user_memory` tool to store that information. Just let me know.



You →  who is Trump?


...

Agent → David Beckham is a former professional soccer player, widely regarded as one of the greatest set-piece specialists and crossers in the game. He played for several top-tier clubs, including Manchester United, Real Madrid, and Paris Saint-Germain, and was also a key player for the English national team. After retiring from football, Beckham has remained a prominent figure, involved in various philanthropic and entrepreneurial endeavors. He's also known for his fashion sense and has been a style icon for many years. Would you like to know more about his career, personal life, or something else?



You →  what are the questions I asked you?


Agent → You asked me:

1. "hey" (a greeting)
2. "who is Trump?" (a question about Donald Trump)
3. "and who is David Backham" (a question about David Beckham, although there was a small typo in the name)

Let me know if you'd like to ask more questions or discuss something specific!



You →  q


Bye! 👋


In [47]:
# chat_loop(agent)

Who are you? ['Nayeem', 'NoMan'] →  NoMan



👋  Chatting as **NoMan**  (type 'exit' to quit)



You →  what are the questions I asked you?


Agent → You haven't asked me any questions yet, this is the start of our conversation. I'm ready to chat and help with any questions you may have. 

To confirm, I haven't stored any memories of our conversation yet, but I can do so using the `update_user_memory` function if needed.



You →  who is Neil?


...

Agent → Now, about Harry Potter: 
Harry Potter is a beloved book series by J.K. Rowling that follows the adventures of a young wizard named Harry Potter and his friends at Hogwarts School of Witchcraft and Wizardry. The series includes seven fantasy novels, with the first book published in 1997 and the last in 2007. The books have been translated into over 80 languages and have sold more than 500 million copies worldwide, making it one of the best-selling book series of all time.

Would you like to know more about the wizarding world, its characters, or something specific?

"Magic is all around, you just have to believe!"



You →  who is Neil?


Agent → According to our conversation, Neil is a person who is known to be dishonest, as you mentioned earlier. I have also stored this information in my memory using the `update_user_memory` function.

" Memories can be tricky, but honesty is always the best policy!"



You →  q


Bye! 👋
